This workbook is a workaround as the AEMDA data is incorrect (TechIDs are mixed up).

This workbook takes the vkm table (from TEAM-Kenya) and the technology table. It 'refreshes' the techIDs.

*WARNING WARNING* this is quick + dirty. Waiting for Paschal's analysis.

In [4]:
import pandas as pd

In [5]:
vkm = pd.read_csv('./ccg_team_notebook/out_data/processed/vkm.csv')
vkm

,Unnamed: 0,ScenarioID,CountryID,TechID,JSTypeVSMID,TechDescription,TransTypeID,VehTypeID,MassCatID,VKM
0,0,1,9,70,1,Road-Small (car)-Conventional (pre-EURO)-Passe...,1,3,1,0.097499
1,1,1,9,70,2,Road-Small (car)-Conventional (pre-EURO)-Passe...,1,3,1,0.048749
2,2,1,9,70,3,Road-Small (car)-Conventional (pre-EURO)-Passe...,1,3,1,0.048749
3,3,1,9,9070,1,Road-Small (car)-Conventional (pre-EURO)-Passe...,1,3,1,0.552493
4,4,1,9,9070,2,Road-Small (car)-Conventional (pre-EURO)-Passe...,1,3,1,0.276247
...,...,...,...,...,...,...,...,...,...,...
1123,2665,1,9,4320,13,Air - Aeroplane - Supersonic - Liquefied Hydro...,1,8,45,0.000000
1124,2666,1,9,4321,9,Air - Aeroplane - Supersonic - Liquefied Hydro...,1,8,45,0.000000
1125,2667,1,9,4321,13,Air - Aeroplane - Supersonic - Liquefied Hydro...,1,8,45,0.000000
1126,2668,1,9,4322,9,Air - Aeroplane - Supersonic - Liquefied Hydro...,1,8,45,0.000000


In [6]:
technology = pd.read_excel(f'./ccg_team_notebook/sample_data/team_kenya_technology.xlsx')
technology

,TechID,ModeID,VehTypeID,MassCatID,FuelID,EngineID,TransTypeID,Availability,Final_Year,HybridFlag,spec_energ_consump,spec_energ_electric,noise_class,PurchasePrice,ExpectedLifeFactor,Capacity,AveEconLife,SecondHandImportFlag
0,10,1,2,4,1,10,1,1980,2000.0,0,6.571951,NaN,0,5000.0,1,2.0,10,0
1,20,1,2,4,12,10,1,1995,2010.0,0,9.514442,NaN,2,7000.0,1,2.0,10,0
2,25,1,2,4,12,11,1,2010,2020.0,0,8.380919,NaN,2,6000.0,1,2.0,10,0
3,26,1,2,4,12,12,1,2020,NaN,0,7.183646,NaN,2,5500.0,1,2.0,10,0
4,30,1,2,4,1,11,1,2000,2010.0,0,5.645645,NaN,0,5000.0,1,2.0,10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2241,14889,1,5,28,13,13,2,2033,2038.0,0,179.200000,1.0,2,14000.0,1,2.5,10,1
2242,14890,1,5,28,13,13,2,2038,2043.0,0,153.890000,1.0,2,12000.0,1,2.5,10,1
2243,14891,1,5,28,13,13,2,2043,2048.0,0,132.150000,1.0,2,12000.0,1,2.5,10,1
2244,14892,1,5,28,13,13,2,2048,2053.0,0,113.480000,1.0,2,12000.0,1,2.5,10,1


In [7]:
for i in list(vkm.index.values):

    # fetch parameters from this row
    techid, transtype, vehtype, masscat, = vkm.TechID[i], vkm.TransTypeID[i], vkm.VehTypeID[i], vkm.MassCatID[i]

    # return matched records in technology
    tech = technology[(technology.TransTypeID == transtype) & (technology.VehTypeID == vehtype) & (technology.MassCatID == masscat)]

    if techid not in tech.TechID.tolist():  # then there is a mismatch

        if not tech.empty:

            if techid > 9000 and not tech[tech.TechID > 9000].empty:
                vkm.at[i, 'TechID'] = tech[tech.TechID > 9000].TechID.tolist()[0]

            else:
                vkm.at[i, 'TechID'] = tech.TechID.tolist()[0]

        else:  # it is empty so find a simpler match

            tech_simple = technology[(technology.VehTypeID == vehtype) & (technology.MassCatID == masscat)]

            if not tech_simple.empty:

                if techid > 9000 and not tech_simple[tech_simple.TechID > 9000].empty:
                    vkm.at[i, 'TechID'] = tech_simple[tech_simple.TechID > 9000].TechID.tolist()[0]

                else:
                    vkm.at[i, 'TechID'] = tech_simple.TechID.tolist()[0]

            else:  # then god help us all just go for vehicle and fuel

                tech_super_simple = technology[(technology.VehTypeID == vehtype)]

                if not tech_super_simple.empty:

                    if techid > 9000 and not tech_super_simple[tech_super_simple.TechID > 9000].empty:
                        vkm.at[i, 'TechID'] = tech_super_simple[tech_super_simple.TechID > 9000].TechID.tolist()[0]

                    else:
                        vkm.at[i, 'TechID'] = tech_super_simple.TechID.tolist()[0]

                else:  # then wtf

                    vkm.at[i, 'TechID'] = technology[technology.VehTypeID==vehtype].TechID.tolist()[0]

vkm

,Unnamed: 0,ScenarioID,CountryID,TechID,JSTypeVSMID,TechDescription,TransTypeID,VehTypeID,MassCatID,VKM
0,0,1,9,70,1,Road-Small (car)-Conventional (pre-EURO)-Passe...,1,3,1,0.097499
1,1,1,9,70,2,Road-Small (car)-Conventional (pre-EURO)-Passe...,1,3,1,0.048749
2,2,1,9,70,3,Road-Small (car)-Conventional (pre-EURO)-Passe...,1,3,1,0.048749
3,3,1,9,9070,1,Road-Small (car)-Conventional (pre-EURO)-Passe...,1,3,1,0.552493
4,4,1,9,9070,2,Road-Small (car)-Conventional (pre-EURO)-Passe...,1,3,1,0.276247
...,...,...,...,...,...,...,...,...,...,...
1123,2665,1,9,4320,13,Air - Aeroplane - Supersonic - Liquefied Hydro...,1,8,45,0.000000
1124,2666,1,9,4321,9,Air - Aeroplane - Supersonic - Liquefied Hydro...,1,8,45,0.000000
1125,2667,1,9,4321,13,Air - Aeroplane - Supersonic - Liquefied Hydro...,1,8,45,0.000000
1126,2668,1,9,4322,9,Air - Aeroplane - Supersonic - Liquefied Hydro...,1,8,45,0.000000


In [8]:
vkm.to_csv(f'./ccg_team_notebook/out_data/processed/afema_bodge/vkm_afemabodge.csv')